## TODO:
- Fix BPM feature, use pyrekordbox library. Requires installation of rekordbox 5.
- Download non proper techno playlists, save them to a df, union PT and nonPT dfs, standardize the data, apply PCA on the mfccs features, train the model -> change download directory in deemix gui to Nonpropertechno folder on hardrive, after generating the csv move it locally to the data folder.

# Creating the proper techno dataset

Assuming we have downloaded the spotify playlists for Proper and NonProper Techno using deemix by pasting the spotify playlist URL in deemix, in this notebook we organize the audio files by class into a pandas dataframe. We are going to download the proper techno spotify playlist and use that as the proper techno true labels. For the negative classes we will take some hard techno, trance, melodic techno spotify playlists as the negatives


In [4]:
import os
# !pwd
# os.chdir("..") # need to cd into the directory first..
# os.chdir("data") # need to cd into the directory first..
# !pwd
# os.chdir("../../../..")
# !pwd
os.chdir("/Volumes/Seagate Expansion Drive/ProperTechno050323/r_ProperTechno")
!pwd

/Volumes/Seagate Expansion Drive/ProperTechno050323/r_ProperTechno


In [ ]:
import os
os.chdir("proper-classifier/data/") # need to cd into the directory first..


In [5]:
import pandas as pd
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline


## EDA

In [ ]:
# testing librosa library on one audio file, setting the params for reading each audio file
# os.chdir("ProperTechno/")
# !pwd
file_name = 'Volster - EXPOSITION F.mp3'
audio_data, sampling_rate = librosa.load(file_name, mono=True) # Mono param here
librosa.display.waveshow(audio_data, sr=sampling_rate)
ipd.Audio(file_name)

In [ ]:
audio_data

In [ ]:
mfccs = librosa.feature.mfcc(y=audio_data, sr=sampling_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs.T,axis=0)

In [ ]:
mfccs_scaled_features

<p> Issue: dealing with Mono is more straightforward but stereo provides more information. The problem is when we do stereo it is not immediately clear how we process the features because we have one set of features for each channel. Do we combine the values together into a 1d array by adding the features in this order ch1,ch2,ch1,ch2? Potentially we could try an initial run and training of the model in mono then try doing it in stereo and see if it makes a difference.<p> 

## Extract the features over each file and create pandas dataframe with: filename (id), features, label

In [ ]:
# Deprecate
def features_extractor(file):
    '''
    Function that takes as input an audio file and extracts the mfccs features + scales the values
    Use in a for loop to extract and scale the features for all tracks in the dataset
    '''
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', mono=False) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40, )
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

# Loop through the ProperTechno folder and the nonpropertechno folder and extract the features for all tracks
# Add them to the pandas data frame with [track title, features, label]
# can do it sequentially starting with ProperTechno tracks, add to DF, then repeat for nonpropertechno and add to df

#Code from the tutorial, needs to be adapted to my custom dataset.
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

    
extracted_features_df=pd.DataFrame(extracted_features,columns=['track','feature','class'])
extracted_features_df.head(10)

# Getting the features

We are computing 40 MFCCs, the tempo and beat features of the track, and the Spectral Centroid and Bandwith using librosa.

We extract the mean spectral centroid, mean spectral bandwidth, and mean zero crossing rate with np.mean() to get a single value for each feature per track. This is a common approach for summarizing frame-level features across time.

## Dev notes

Skipping tempo calculation using librosa as is not correct due to the algorithm not being optimized for electronic music. Rekordbox's algorithm is much better suited for our case. Not available through an API though, so need to use this library https://github.com/dylanljones/pyrekordbox

In [6]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Set the directory containing the audio files
audio_dir = "ProperTechno/"

# Initialize an empty list to store the data
data = []

# Loop through each file in the directory
for subdir, dirs, files in os.walk(audio_dir):
    for file in files:
        # Load the audio file
        file_path = os.path.join(subdir, file)
        print(file)
        y, sr = librosa.load(file_path, mono=True, duration=30)
        
        parent_dir = os.path.basename(os.path.dirname(file_path))

        # Extract the MFCC, spectral centroid and bandwidth, beat features, and zero crossing rate features
        
        #MFCCs features - Decrease the n_mfcc param?
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20, hop_length=512)
        
#         print("MFCCs Shape: {}".format(mfccs.shape))
        
        # Spectral features
        spec_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spec_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        
        # Beat features
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        beat_times = librosa.frames_to_time(beats, sr=sr)
        ibi = np.diff(beat_times)
        ibi_mean = np.mean(ibi)
        ibi_min = np.min(ibi)
        ibi_max = np.max(ibi)
        
        # Zero Crossing Rate feature
        zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))
        
        # Create a row of data with the filename, MFCCs, spectral centroid, spectral bandwidth, and beat features
        row = [file, parent_dir]
        
#         print("To list shape: {}".format(np.array(mfccs.tolist()).shape))
#         print("Flattened to list shape: {}".format(np.array(mfccs.flatten().tolist()).shape))

        row.extend(mfccs.flatten().tolist())
        
        row.append(spec_centroid)
        row.append(spec_bandwidth)
        row.append(ibi_mean)
        row.append(ibi_min)
        row.append(ibi_max)
        row.append(zcr)

        # Append the row to the data list
        data.append(row)

# Create a dataframe from the data list
columns = ["track", "label"]
for i in range(25840):
    columns.append("mfccs_feature_" + str(i+1))
columns.append("spec_centroid")
columns.append("spec_bandwidth")
columns.append("ibi_mean")
columns.append("ibi_min")
columns.append("ibi_max")
columns.append("zcr")

df = pd.DataFrame(data, columns=columns)
df

1736 - Ignez & Rødhåd - VERMILLION 03 [220208.2].mp3
1737 - Ignez & Rødhåd - VERMILLION 04 [220208.1].mp3
1739 - MoMa Ready - Tech-Nic.mp3
1740 - Samuli Kemppi - Seeker (Original Mix).mp3
1741 - Tripeo - Anipintiros #5 + #8 (Exium Mash Up Remix).mp3
1742 - Baby Ford - Dead Eye.mp3
1743 - Marco Bailey - Attor (Original Mix).mp3
1744 - Joey Beltram - Cop Car.mp3
1745 - Bandulu - Wetlook.mp3
1746 - Temudo - You Spelled Corn Wrong.mp3
1747 - Coefficient - Neutron Star Remnant.mp3
1748 - Dissembler - Oleander.mp3
1749 - Pau Cabeza - Summit.mp3
1750 - Sterac - Darkness in My Life.mp3
1751 - Robert Natus - Day After.mp3
1752 - Alexander Johansson - Blaenk (Original Version).mp3
1753 - Kenny Larkin - Maritime.mp3
1754 - Blawan - Hapexil Rotator.mp3
1755 - Alexander Johansson & Mattias Fridell - Sista Ordet.mp3
1756 - Rolando - Wheesht (Ben Sims Remix) (Ben Sims).mp3
0001 - Allan pillai - Twine.mp3
0002 - Oliver Ho - Mutate & Survive.mp3
0003 - Gotshell - Syndrome (Takaaki Itoh Remix).mp3
000

0197 - ANDC - Intense Reaction.mp3
0198 - Rino Cerrone - Optical Way A1.mp3
0199 - The 65D Mavericks - In the Dark.mp3
0200 - Internal Explorer - Stringent.mp3
0201 - Ketch - Eternal 2.mp3
0202 - CW_A - Day Of Riddance.mp3
0203 - MSDMNR - First Signal.mp3
0204 - Glenn Wilson - Conclusion.mp3
0205 - Oliver Kapp - Treeday (Grindvik's 2013 Rework).mp3
0206 - Phyxix - Depths.mp3
0207 - Rene Wise - Swamp Dancer.mp3
0209 - BW - Maña.mp3
0210 - LYRIC (GER) - Declination (Syva001).mp3
0212 - Cari Lekebusch - Flaunt (Original Mix).mp3
0213 - PUSHMANN - Mirage II.mp3
0214 - SSTROM - Volatile Substance.mp3
0215 - Ocherii - Non Finito (Original Mix).mp3
0216 - Altinbas - Flows.mp3
0217 - Cirkle - Unkulu Tribe.mp3
0218 - Internal Explorer - Rigorous (Alarico Remix).mp3
0219 - Angioma - Break Your Chains.mp3
0220 - Hattori Hanzo - Radiation Problem.mp3
0221 - Sev Dah - 152.mp3
0223 - Unis - 14.II.1945.mp3
0224 - Morrice - Synth Lies.mp3
0225 - Wice (DE) - Ape Layers (Original).mp3
0226 - Eric Sneo 

0422 - Masseran - Dune Swing (Original Mix).mp3
0423 - Border One - Cyclone.mp3
0424 - Echologist - Reanimation.mp3
0425 - Slone - Lancer.mp3
0426 - Pfirter & Oliver Rosemann - Beta 02.mp3
0427 - LaChriz - Truthery (Marco Zenker Remix).mp3
0428 - Redshape - Phonkin.mp3
0429 - MAL_HOMBRE - CONSTALATION TRAJECTORY.mp3
0430 - Uncertain - Grind.mp3
0431 - PTTRN - Shoals 3.mp3
0433 - Various Artists - Token Introspective (Mixed by Kr!z).mp3
0434 - Antonio De Angelis - AYD (Original Mix).mp3
0435 - Jim Solis - Wild Fire (Danitz Remix).mp3
0436 - No.name - O'clock.mp3
0437 - Israel Toledo - Abstrakt.mp3
0439 - Regent - Aftermath.mp3
0440 - Axel Karakasis - Breeze of Flame.mp3
0441 - Ruben Ganev - Quantum.mp3
0442 - B. Riley - No Good Deed.mp3
0443 - Lidvall - Legal violence (Original mix).mp3
0444 - Neznan - Rotor (Original Mix).mp3
0158 - Temudo - Always Does Until It Doesn't (Original mix).mp3
0171 - Monobox - Forwardbase Kodai (Robert Hood Re-Plant).mp3
0184 - Slam - SD 00.04.mp3
0196 - Tr

0628 - KaioBarssalos - Broken Love.mp3
0629 - Axel Karakasis - Animations.mp3
0630 - Skov Bowden - Venom.mp3
0631 - Robert Natus - Hermoso (Tribal Version).mp3
0633 - sleeparchive - 1.mp3
0634 - Setaoc Mass - M1.mp3
0635 - Truncate - Modify.mp3
0636 - Fhase 87 - French Loops 02.B.mp3
0637 - Fadi Mohem - We Are.mp3
0640 - Aske - Xólotl.mp3
0641 - Ignez - Anomaly.mp3
0642 - sleeparchive - Session Seven.mp3
0643 - CNCPT - 000.1.mp3
0644 - Wrong Assessment - The Astrologist.mp3
0645 - Jen Series - Angustias.mp3
0646 - Akilah Bryant - Ruff Sex.mp3
0647 - Hedström & Pflug - Irrtum.mp3
0649 - Devilfish - Jammin'.mp3
0650 - Alarico - Running Through.mp3
0651 - sleeparchive - Trust (V.2).mp3
0652 - Arthur Robert - Relentless.mp3
0653 - Marçal - The Only Way is Forward.mp3
0654 - Alessandro Grops - Sharp.mp3
0655 - Elisa Bee - Way Back.mp3
0656 - Alessandro Grops - Toolbox.mp3
0657 - Jack Fresia - MFZ.mp3
0658 - RONY Group - South Side.mp3
0659 - Tensal - Strobo.mp3
0660 - Marçal - Oboro.mp3

0853 - Chlär - Occult Authority.mp3
0854 - Truncate - Basal 4.mp3
0855 - Rill - Sunil.mp3
0856 - Øblak - SINCOPE (Original Mix).mp3
0857 - Ketch - Traffic (Original Mix).mp3
0858 - exos - Áttfalt (Octal Industries Remix).mp3
0859 - Memorial Home - Acoustic Treatment.mp3
0860 - ИAИO - Distorted Light (Original Mix).mp3
0861 - Cvrdwell - Hellraiser (Original Mix).mp3
0863 - Pyramidal Decode - Axes.mp3
0864 - Infiniti, Juan Atkins - Think Quick (Sleeparchive Remix).mp3
0865 - Franz Jäger - Floating Point.mp3
0866 - ISH11 - FLUID2.mp3
0867 - Hyden - Scratches on wax (Live).mp3
0868 - Jacobworld - Malice.mp3
0869 - Mesh Convergence - 1997.mp3
0870 - Fireground - Dreams.mp3
0871 - Mark Broom - Disco Story.mp3
0872 - Marcelo Antonio - Chronicles I.mp3
0873 - Yan Cook - Counterclockwise.mp3
0875 - Nicolas Vogler - Power Cut (Original Mix).mp3
0876 - Linkan Ray - Planes.mp3
0877 - Oat M - Metashake.mp3
0878 - Rolando - The Afterlife.mp3
0880 - Stier - Mutation (Vegim Repaint).mp3
0881 - Samu

1061 - Encoder - Untitled 02 29 (Original Mix).mp3
1062 - Allan Gallego - XX (Oliver Deutschmann Remix).mp3
1063 - Atonism - Timetravel.mp3
1065 - No.name - Smoke.mp3
1066 - Deniro - Vaquita.mp3
1068 - Judy - Gaueko VIII.mp3
1069 - Obscurum - Tul.mp3
1070 - Shallowed - Space 79.mp3
1071 - ToG - DoctorRobert.mp3
1072 - Sanna Mun - Chiaroscuro.mp3
1073 - Slone - Slone (Ghost).mp3
1074 - Sev Dah - Seven Brothers.mp3
1075 - Tensal - Magnitude 1.mp3
1076 - Rngd - 40 (Original Mix).mp3
1077 - Toru Ikemoto - %038.mp3
1078 - Claude Young - Ancelyn.mp3
1080 - Ricardo Garduno - A Very Short Memory.mp3
1081 - Decoder - Rova.mp3
1082 - Yan Cook - Cold Spots.mp3
1083 - Roogunit - Edge 3.mp3
1084 - Marco Bruno - Detached.mp3
1085 - Floorplan - Never Grow Old (Re-Plant).mp3
1086 - Brunson - Hoffman's S.O.S. (Scully, Owsley, Sand).mp3
1087 - Son Error - Strange Man (At The Corner).mp3
1088 - Son Error - Lichtenberg Mambo.mp3
1089 - Son Error - Creeping Spirits.mp3
1090 - Son Error - Change Of Characte

1283 - Amorphic - A70 (v2).mp3
1284 - D.Dan - Vaults.mp3
1286 - Rolando - Juu (Jonas Kopp Remix).mp3
1287 - Jeroen - Vespertine (Deniro's Mix).mp3
1288 - Jack Fresia - Gisko.mp3
1289 - Ian Void - The Rascal (Chris McCormack Remix).mp3
1290 - Talismann - Fryction.mp3
1291 - Beste Hira - Sail.mp3
1292 - Mion - Fused.mp3
1293 - Leo Anibaldi - Muta 5 (Donato Dozzy Remix).mp3
1294 - Talismann - Anchoryst.mp3
1295 - T.I.E.R - Spielplatz.mp3
1296 - Zisko - Merciless Decisions.mp3
1297 - Iglo - Dissolution.mp3
1299 - William Arist - Excesivo.mp3
1300 - 1fm - Soul Overload (Truncate Remix).mp3
1301 - Lewis Fautzi - Murder The Limits.mp3
1302 - Hohtogrammi - Breaking Points.mp3
1303 - Antonio De Angelis - SSSS (Original Mix).mp3
1304 - Inigo Kennedy - Rotator Cuff.mp3
1305 - Developer - Fake Artesians.mp3
1306 - Franco Rossi - Festina Lente.mp3
1307 - D-Source - Palace (Original Mix).mp3
1308 - PS Eleven - Unrest (Original Mix).mp3
1309 - Gotshell - Alucinaciones.mp3
1311 - Connor Wall - Fusion 

1498 - Fixon - Top To Bottom (Oliver Rosemann Remix).mp3
1499 - Modem - Indigo (Maccari Remix).mp3
1500 - Damon Wild - 13 (Year Old Girl).mp3
1501 - Enko - Opheela.mp3
1502 - Peter Van Hoesen - Ascending.mp3
1503 - Augusto Taito - Peoplar.mp3
1504 - A.Morgan - Omega.mp3
1505 - Tadeo - Futura Conga (Orbe Remix).mp3
1506 - Marcelo Antonio - Neil or Lance.mp3
1507 - Phyxix - The Nest.mp3
1509 - Kr!z - The Tunnel.mp3
1510 - Temudo - 2023.mp3
1511 - Linear System - Acacia.mp3
1512 - Deniro - Atavism.mp3
1513 - DJ Dextro - Alcides.mp3
1514 - Bidoben - Gstaad.mp3
1515 - dxrvo - Extase.mp3
1516 - Matias Freccia - Evol (Original Mix).mp3
1517 - Uncertain - Suspect.mp3
1518 - Alexskyspirit - Existence (Original mix).mp3
1519 - Enko - Ikigai.mp3
1520 - Sterac - Intersphere (mixed).mp3
1522 - Robert Hoff - Codification.mp3
1523 - Rosati - Lexicon.mp3
1524 - Audio Units - Globular Cluster (Original Version).mp3
1525 - Minus Magnus - Cynical Romance (Skodde Remix).mp3
1526 - Takt - Botanic.mp3
1527 

1729 - Robert Natus - Eisga.mp3
1730 - Age - Technik.mp3
1731 - Samuel L Session & Van Czar - Marked Spot.mp3
1732 - Stephen Brown & Fossil Archive aka Roberto - 3D World.mp3
1733 - Modern Doom - Neon God.mp3
1734 - Ignez & Rødhåd - VERMILLION 01 [220207.1].mp3


,track,label,mfccs_feature_1,mfccs_feature_2,mfccs_feature_3,mfccs_feature_4,mfccs_feature_5,mfccs_feature_6,mfccs_feature_7,mfccs_feature_8,...,mfccs_feature_25837,mfccs_feature_25838,mfccs_feature_25839,mfccs_feature_25840,spec_centroid,spec_bandwidth,ibi_mean,ibi_min,ibi_max,zcr
0,1736 - Ignez & Rødhåd - VERMILLION 03 [220208...,ProperTechno,-463.512939,-463.512939,-463.512939,-463.512939,-463.512939,-463.512939,-463.512939,-368.316223,...,14.788702,17.027441,14.987085,12.223713,1108.425686,1709.898955,0.428703,0.417959,0.441179,0.016021
1,1737 - Ignez & Rødhåd - VERMILLION 04 [220208...,ProperTechno,-466.781097,-466.781097,-466.781097,-466.781097,-466.781097,-466.781097,-466.781097,-303.010864,...,6.824379,8.259930,4.222588,1.606868,1348.034435,2099.686805,0.444444,0.441179,0.464399,0.020412
2,1739 - MoMa Ready - Tech-Nic.mp3,ProperTechno,-43.770481,-38.471272,-103.160294,-150.575287,-160.435028,-126.830070,-106.481354,-127.923889,...,7.103097,6.492336,2.266473,3.314648,3314.002146,2982.679417,0.444394,0.441179,0.464399,0.155759
3,1740 - Samuli Kemppi - Seeker (Original Mix).mp3,ProperTechno,-458.259216,-264.199921,-90.370201,-77.227394,-187.913284,-330.891174,-381.720428,-392.016418,...,12.054602,11.683971,13.779953,10.830130,3361.674501,3534.660341,0.458231,0.441179,0.464399,0.096435
4,1741 - Tripeo - Anipintiros #5 + #8 (Exium Mas...,ProperTechno,-151.503082,-117.933578,-208.448792,-317.158844,-342.284027,-341.079987,-325.770020,-320.847321,...,-7.848754,-4.057745,-5.996547,-4.487836,298.607432,790.200202,0.469043,0.464399,0.487619,0.009257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,1730 - Age - Technik.mp3,ProperTechno,-502.502991,-202.400726,-106.510460,-131.665817,-236.697693,-287.501495,-253.224762,-252.102463,...,1.162379,-0.292377,1.045823,-0.648997,1128.392013,1997.069369,0.538878,0.534059,0.650159,0.013158
1676,1731 - Samuel L Session & Van Czar - Marked Sp...,ProperTechno,-452.727875,-452.397186,-452.366882,-452.357300,-452.251282,-452.323303,-324.841339,-85.818146,...,13.427003,1.766355,4.199123,5.667727,1417.479586,2002.784653,0.447609,0.441179,0.464399,0.026381
1677,1732 - Stephen Brown & Fossil Archive aka Robe...,ProperTechno,-501.857544,-501.857544,-325.433319,-92.156082,-47.145752,-99.275230,-170.070938,-212.400818,...,2.207502,2.315609,8.300816,8.235739,1676.499912,2293.575829,0.581892,0.510839,0.696599,0.036360
1678,1733 - Modern Doom - Neon God.mp3,ProperTechno,-451.107117,-94.778419,-18.187366,-56.561562,-163.979416,-218.515320,-148.152542,-133.100891,...,0.725851,2.958272,1.132126,0.750343,1297.088983,1997.855730,0.438060,0.417959,0.441179,0.025926


In [7]:
# Save the dataframe to a CSV file
df.to_csv("df_track_features.csv", index=False)

In [12]:
os.chdir("/Users/zein/proper-classifier/data")
!pwd


/Users/zein/proper-classifier/data


In [13]:
df = pd.read_csv('df_track_features.csv')
df

,track,label,mfccs_feature_1,mfccs_feature_2,mfccs_feature_3,mfccs_feature_4,mfccs_feature_5,mfccs_feature_6,mfccs_feature_7,mfccs_feature_8,...,mfccs_feature_25837,mfccs_feature_25838,mfccs_feature_25839,mfccs_feature_25840,spec_centroid,spec_bandwidth,ibi_mean,ibi_min,ibi_max,zcr
0,1736 - Ignez & Rødhåd - VERMILLION 03 [220208...,ProperTechno,-463.512939,-463.512939,-463.512939,-463.512939,-463.512939,-463.512939,-463.512939,-368.316223,...,14.788702,17.027441,14.987085,12.223713,1108.425686,1709.898955,0.428703,0.417959,0.441179,0.016021
1,1737 - Ignez & Rødhåd - VERMILLION 04 [220208...,ProperTechno,-466.781097,-466.781097,-466.781097,-466.781097,-466.781097,-466.781097,-466.781097,-303.010864,...,6.824379,8.259930,4.222588,1.606868,1348.034435,2099.686805,0.444444,0.441179,0.464399,0.020412
2,1739 - MoMa Ready - Tech-Nic.mp3,ProperTechno,-43.770481,-38.471272,-103.160294,-150.575287,-160.435028,-126.830070,-106.481354,-127.923889,...,7.103097,6.492336,2.266473,3.314648,3314.002146,2982.679417,0.444394,0.441179,0.464399,0.155759
3,1740 - Samuli Kemppi - Seeker (Original Mix).mp3,ProperTechno,-458.259216,-264.199921,-90.370201,-77.227394,-187.913284,-330.891174,-381.720428,-392.016418,...,12.054602,11.683971,13.779953,10.830130,3361.674501,3534.660341,0.458231,0.441179,0.464399,0.096435
4,1741 - Tripeo - Anipintiros #5 + #8 (Exium Mas...,ProperTechno,-151.503082,-117.933578,-208.448792,-317.158844,-342.284027,-341.079987,-325.770020,-320.847321,...,-7.848754,-4.057745,-5.996547,-4.487836,298.607432,790.200202,0.469043,0.464399,0.487619,0.009257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,1730 - Age - Technik.mp3,ProperTechno,-502.502991,-202.400726,-106.510460,-131.665817,-236.697693,-287.501495,-253.224762,-252.102463,...,1.162379,-0.292377,1.045823,-0.648997,1128.392013,1997.069369,0.538878,0.534059,0.650159,0.013158
1676,1731 - Samuel L Session & Van Czar - Marked Sp...,ProperTechno,-452.727875,-452.397186,-452.366882,-452.357300,-452.251282,-452.323303,-324.841339,-85.818146,...,13.427003,1.766355,4.199123,5.667727,1417.479586,2002.784653,0.447609,0.441179,0.464399,0.026381
1677,1732 - Stephen Brown & Fossil Archive aka Robe...,ProperTechno,-501.857544,-501.857544,-325.433319,-92.156082,-47.145752,-99.275230,-170.070938,-212.400818,...,2.207502,2.315609,8.300816,8.235739,1676.499912,2293.575829,0.581892,0.510839,0.696599,0.036360
1678,1733 - Modern Doom - Neon God.mp3,ProperTechno,-451.107117,-94.778419,-18.187366,-56.561562,-163.979416,-218.515320,-148.152542,-133.100891,...,0.725851,2.958272,1.132126,0.750343,1297.088983,1997.855730,0.438060,0.417959,0.441179,0.025926


## Extract the BPM from Rekorbox

To extract the BPM value that Rekordbox calculated, we first need to download the playlist locally, then import it in a rekordbox playlist.

In [3]:
from pyrekordbox import show_config
from pyrekordbox.config import update_config

show_config()

# need to install rekordbox 5

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
import pyrekordbox
import json

# Connect to Rekordbox
rekordbox = pyrekordbox.Rekordbox()

# Set the name of the playlist to analyze
playlist_name = "My Playlist"

# Get the tracks in the playlist
playlist_tracks = rekordbox.get_playlist_tracks(playlist_name)

# Loop through the tracks and get the BPM
for track in playlist_tracks:
    # Get the track info
    track_info = rekordbox.get_track_info(track['id'])
    
    # Get the BPM from the track info
    bpm = track_info.get('TEMPO', {}).get('value')
    
    # Print the track name and BPM
    print(f"{track['name']} - BPM: {bpm}")

# Disconnect from Rekordbox
rekordbox.disconnect()


# Preprocessing (Dim reduct, and standardization)

We want to scale the features in the df appropriately here to normalize everything.

In [ ]:
#standardize all the features


# Perform PCA on the standardized MFCC features
pca = PCA(n_components=40)
principal_components = pca.fit_transform(mfcc_features)

# Add the principal components back to the dataframe
for i in range(num_components):
    data[f'PC{i+1}'] = principal_components[:,i]